In [1]:
import os
import h5py
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.features import shapes
from datetime import datetime, timedelta
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor


In [2]:
def extract_timestamp(filename):
    parts = filename.split('.')
    year_day = parts[1][1:]  # Example: '2000058'
    time = parts[2]  # Example: '0515'

    year = int(year_day[:4])
    day_of_year = int(year_day[4:])
    dt = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)

    hour = int(time[:2])
    minute = int(time[2:4])
    return datetime(year, dt.month, dt.day, hour, minute)

In [1]:
def process_modis_file(file_path):
    print(f"Processing file: {file_path}")
    timestamp = extract_timestamp(os.path.basename(file_path))
    
    try:
        with h5py.File(file_path, 'r') as file:
            latitude = file['FP_latitude'][()]
            longitude = file['FP_longitude'][()]
            t21 = file['FP_T21'][()]
            t31 = file['FP_T31'][()]
            power = file['FP_power'][()]
            confidence = file['FP_confidence'][()]

            # Convert data to 'native' endianness if necessary
            if latitude.dtype.byteorder == '>':
                latitude = latitude.byteswap().newbyteorder()
            if longitude.dtype.byteorder == '>':
                longitude = longitude.byteswap().newbyteorder()
            if t21.dtype.byteorder == '>':
                t21 = t21.byteswap().newbyteorder()
            if t31.dtype.byteorder == '>':
                t31 = t31.byteswap().newbyteorder()
            if power.dtype.byteorder == '>':
                power = power.byteswap().newbyteorder()
            if confidence.dtype.byteorder == '>':
                confidence = confidence.byteswap().newbyteorder()

            modis_df = pd.DataFrame({
                'Latitude': latitude.flatten(),
                'Longitude': longitude.flatten(),
                'T21': t21.flatten(),
                'T31': t31.flatten(),
                'Power': power.flatten(),
                'Confidence': confidence.flatten(),
                'Timestamp': timestamp  # Add timestamp to each row
            })

            # Drop rows with invalid coordinates
            modis_df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

            # Convert to a GeoDataFrame
            geometry = [Point(xy) for xy in zip(modis_df.Longitude, modis_df.Latitude)]
            gdf_modis = gpd.GeoDataFrame(modis_df, geometry=geometry)

            # Set the CRS for MODIS data here
            gdf_modis.set_crs(epsg=4326, inplace=True)

            return gdf_modis

    except OSError as e:
        print(f"Error processing file {file_path}: {e}")
        return None


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/test_data.csv'

In [4]:
def process_gap_raster(file_path):
    with rasterio.open(file_path) as src:
        band1 = src.read(1)
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) in enumerate(shapes(band1, mask=None, transform=src.transform))
            if v != src.nodatavals[0]
        )
        gdf = gpd.GeoDataFrame.from_features(results)
        gdf.crs = src.crs
        return gdf

def rasterize_points(gdf, resolution_x, resolution_y, bounds):
    min_x, min_y, max_x, max_y = bounds
    raster = np.zeros((resolution_y, resolution_x))
    cell_size_x = (max_x - min_x) / resolution_x
    cell_size_y = (max_y - min_y) / resolution_y
    for _, point in gdf.iterrows():
        col = int((point.geometry.x - min_x) / cell_size_x)
        row = int((point.geometry.y - min_y) / cell_size_y)
        if 0 <= col < resolution_x and 0 <= row < resolution_y:
            raster[row, col] += 1
    return raster

In [5]:
# Set the directory for saving output files
output_dir = '/Volumes/LaCie/Deep_Learning_Final_Project/Machine/Processed_Data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process GAP raster data
gap_file_path = '/Volumes/LaCie/Deep_Learning_Final_Project/Machine/gaplf2011lc_v30_CA/gaplf2011lc_v30_ca.tif'
gdf_gap = process_gap_raster(gap_file_path)

# Process each MODIS file in parallel
data_dir = '/Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0'
all_files = [f for f in os.listdir(data_dir) if f.endswith('.h5')]


In [6]:
def process_file(file):
    file_path = os.path.join(data_dir, file)
    gdf_modis = process_modis_file(file_path)
    if gdf_modis is not None and not gdf_modis.empty:
        if gdf_modis.crs != gdf_gap.crs:
            gdf_modis = gdf_modis.to_crs(gdf_gap.crs)
        gdf_merged = gpd.sjoin(gdf_modis, gdf_gap, how="inner", predicate='intersects')
        if not gdf_merged.empty:
            timestamp_str = gdf_modis['Timestamp'].iloc[0].strftime('%Y%m%d%H%M')
            output_file_path = os.path.join(output_dir, f'merged_data_{timestamp_str}.geojson')
            gdf_merged.to_file(output_file_path, driver='GeoJSON')
        else:
            print(f"No spatial join results for file {file}")
    else:
        print(f"No valid MODIS data found in file {file}")

def safe_process_file(file):
    try:
        process_file(file)
    except Exception as e:
        print(f"Error processing file {file}: {e}")
        return file
    return None

In [7]:
# Ensure GAP data CRS is set
if gdf_gap.crs is None:
    gdf_gap.set_crs(epsg=4326, inplace=True)  # Assuming GAP data is in WGS 84


# Process each MODIS file in parallel, starting from the last processed file
last_processed_file = 'MOD14.A2008049.1750.061.2021085140006.h5'
all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.h5')])

# Find the index of the last processed file
try:
    last_index = all_files.index(last_processed_file)
except ValueError:
    last_index = -1  # If the file is not found, start from the beginning

# Process files from the next one after the last processed file
files_to_process = all_files[last_index + 1:]

failed_files = []
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(safe_process_file, file): file for file in files_to_process}
    for future in as_completed(futures):
        try:
            result = future.result()
            if result is not None:
                failed_files.append(result)
        except Exception as e:
            print(f"Error processing file: {e}")

Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008049.1925.061.2021085140855.h5Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008050.0450.061.2021085141157.h5

Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008050.0630.061.2021085141752.h5
Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008050.1830.061.2021085162042.h5
No valid MODIS data found in file MOD14.A2008050.0630.061.2021085141752.h5
Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008050.2005.061.2021085164712.h5
No spatial join results for file MOD14.A2008050.0450.061.2021085141157.h5
Processing file: /Volumes/LaCie/Deep_Learning_Final_Project/Machine/Earth_Data2.0/MOD14.A2008050.2010.061.2021085164511.h5
No spatial join results for file MOD14.A2008050.2005.061.2021085164712.h5
Processing file: /Volumes/LaCie/Deep_Le